In [21]:
import yfinance as yf
import pandas as pd

data_inicial = '1994-01-02'
data_final = '2026-01-21'

bvsp = yf.download('^BVSP', start=data_inicial, end=data_final,auto_adjust=True)[['Close']]
fechamento_mensal = bvsp['Close'].resample('MS').last()
# fechamento_mensal

[*********************100%***********************]  1 of 1 completed


In [22]:
import urllib.request
import urllib.error
from io import StringIO
from datetime import datetime



dinicio = datetime.strptime(data_inicial, "%Y-%m-%d").strftime("%d/%m/%Y")
dfim = datetime.strptime(data_final, "%Y-%m-%d").strftime("%d/%m/%Y")

codigo = 433 

url_bcb = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo}/dados?formato=csv&dataInicial={dinicio}&dataFinal={dfim}"


req = urllib.request.Request(url_bcb)
req.add_header('User-Agent', 'Mozilla/5.0')

with urllib.request.urlopen(req, timeout=30) as response:
    content = response.read().decode('utf-8')

serie_ipca = pd.read_csv(StringIO(content), sep=";", decimal=',')

serie_ipca['data'] = pd.to_datetime(serie_ipca['data'], format='%d/%m/%Y')
serie_ipca.set_index('data', inplace=True)



In [27]:

codigo = 190
url_bcb = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo}/dados?formato=csv&dataInicial={dinicio}&dataFinal={dfim}"

req = urllib.request.Request(url_bcb)
req.add_header('User-Agent', 'Mozilla/5.0')

with urllib.request.urlopen(req, timeout=30) as response:
    content = response.read().decode('utf-8')

serie_igpdi = pd.read_csv(StringIO(content), sep=";", decimal=',')


serie_igpdi['data'] = pd.to_datetime(serie_igpdi['data'], format='%d/%m/%Y')
serie_igpdi.set_index('data', inplace=True)


In [29]:
# Merge usando os índices
df = pd.merge(fechamento_mensal, serie_ipca, 
              left_index=True, right_index=True, 
              how='inner')

df = pd.merge(df ,  serie_igpdi, 
              left_index=True, right_index=True, 
              how='inner')
df.columns = ['BVSP', 'IPCA' , 'IGPDI']
print(df.head())
print(f"\nTotal de meses: {len(df)}")

                   BVSP   IPCA  IGPDI
1994-01-01   740.599976  41.31  42.19
1994-02-01  1053.900024  40.27  42.41
1994-03-01  1515.599976  42.75  44.83
1994-04-01  1708.400024  42.68  42.46
1994-05-01  2467.199951  44.03  40.95

Total de meses: 384


In [30]:
df['IPCA_acumulado'] = (1 + df['IPCA']/100).cumprod()
df['IGPDI_acumulado'] = (1 + df['IGPDI']/100).cumprod()

ipca_final  = df['IPCA_acumulado'].iloc[-1]
igpdi_final = df['IGPDI_acumulado'].iloc[-1]


#  BOVESPA corrigido pela inflação (valores reais)
# Trazendo tudo para valores do primeiro mês
df['BVSP_real_IPCA']  = (df['BVSP'] / df['IPCA_acumulado']) * ipca_final
df['BVSP_real_IGPDI'] = (df['BVSP'] / df['IGPDI_acumulado']) * igpdi_final

In [31]:
import plotly.graph_objects as go

fig = go.Figure()

# Adicionar as três linhas
fig.add_trace(go.Scatter(
    x=df.index,y=df['BVSP_real_IPCA'], mode='lines', name='Ajustado pelo IPCA', line=dict(color="#060F63", width=2.5)
))

fig.add_trace(go.Scatter(
    x=df.index, y=df['BVSP_real_IGPDI'], mode='lines',name='Ajustado pelo IGPDI',line=dict(color='#2E5F8A', width=2.5)
))

fig.add_trace(go.Scatter(
    x=df.index, y=df['BVSP'], mode='lines',name='Nominal',   line=dict(color="#830A06", width=2.5)
))

# Configurar layout
fig.update_layout(
    title='Pontuações Ibovespa nominal e ajustadas pela inflação', title_font=dict(size=18, family='Arial', color='black'),
    xaxis_title='Data', yaxis_title='Pontos',
    font=dict(size=12),
    # hovermode='x unified',
    template='plotly_white',
    width=1200,
    height=600,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
        bgcolor="rgba(255, 255, 255, 0.8)"
    )
)

# Configurar grid
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

fig.show()

In [33]:
df['BVSP_real_IPCA']  = (df['BVSP'] / df['IPCA_acumulado'])
df['BVSP_real_IGPDI'] = (df['BVSP'] / df['IGPDI_acumulado']) 

fig = go.Figure()

# Adicionar as três linhas
fig.add_trace(go.Scatter(
    x=df.index,y=df['BVSP_real_IPCA'], mode='lines', name='descontado IPCA', line=dict(color="#060F63", width=2.5)
))

fig.add_trace(go.Scatter(
    x=df.index, y=df['BVSP_real_IGPDI'], mode='lines',name='descontado IGPDI',line=dict(color='#2E5F8A', width=2.5)
))

fig.add_trace(go.Scatter(
    x=df.index, y=df['BVSP'], mode='lines',name='Nominal',   line=dict(color="#830A06", width=2.5)
))

# Configurar layout
fig.update_layout(
    title='Pontuações Ibovespa nominal e ajustadas pela inflação', title_font=dict(size=18, family='Arial', color='black'),
    xaxis_title='Data', yaxis_title='Pontos',
    font=dict(size=12),
    # hovermode='x unified',
    template='plotly_white',
    width=1200,
    height=600,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
        bgcolor="rgba(255, 255, 255, 0.8)"
    )
)

# Configurar grid
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

fig.show()